# Project: Customer Relationship Prediction
Univ.AI <br>
AI-1 Cohort-5

## Name of the people who worked on the project:

Link to presentation: *link here*

Link to slides: *link here*

Link to poster: *link here*

Any other information:*link here*